In [0]:
import keras
keras.__version__

Using TensorFlow backend.


'2.3.1'

In [0]:
#Libraries
import pandas as pd
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy
from spacy.lang.pt.examples import sentences
import matplotlib.pyplot as plt
import scipy.sparse
from sklearn.decomposition import PCA
import pickle
#nlp = spacy.load('pt')
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE 
import matplotlib.pyplot as plt
#pip install scipy==0.10.1
import gensim
from gensim.models import KeyedVectors

from __future__ import division, print_function
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import text
from tensorflow.keras.layers import Bidirectional
from sklearn.model_selection import train_test_split
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
#Take Files From GoogleDrive
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/Research/DataSets/')
path= '/content/drive/My Drive/Research/DataSets/'

In [0]:
%run '/content/drive/My Drive/Fraud detection in internal auditing/Colab Notebooks/NLP functions_Gus.ipynb'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
def overSamplingRISKs(df,rate):
  X = df 
  not_HIGH = df[df.RISK!="HIGH"]
  HIGH = df[df.RISK=="HIGH"]
  oversample_by= int((rate*((len(df[df.RISK!="HIGH"])+len(df[df.RISK=="HIGH"]))-(len(df[df.RISK=="HIGH"]))))/(1-rate))
  # OVERsample minority
  high_upsampled = resample(HIGH,
                            replace=True, # sample with replacement
                            n_samples=oversample_by, # match number in majority class
                            random_state=27) # reproducible results

  upsampled = pd.concat([not_HIGH, high_upsampled])

  print( upsampled.RISK.value_counts() )

  return upsampled

In [0]:
DATASETS_FOLDER = '/content/drive/My Drive/Research/DataSets/'
from sklearn.model_selection import train_test_split
import numpy as np
from keras.layers import MaxPooling1D

ele='2016_DF_cleaned_not_balanced.pkl'
filename = DATASETS_FOLDER+ele
df = pd.read_pickle(filename)
datatrain,datatest= train_test_split(df, test_size=0.30, random_state=42)
datatrain = overSamplingRISKs(datatrain,0.30)

MEDIUM-LOW    12467
HIGH           5343
Name: RISK, dtype: int64


In [0]:
MAX_SEQUENCE_LENGTH = 5000
EMBEDDING_DIM = 300
emdedding_size = 32

In [0]:
# !git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!pip install .
import fasttext
import fasttext.util

/content/drive/My Drive/Research/DataSets/fastText/fastText
Processing /content/drive/My Drive/Research/DataSets/fastText/fastText
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2860074 sha256=c3f260fc118990ba650336fc85f131782082062f9c664cd1cc55f71cd2183d2c
  Stored in directory: /tmp/pip-ephem-wheel-cache-hmalh2a3/wheels/fe/7e/1e/eaf07aad2285f654878beed1cd1d300bd9b0715b3a6e537e7e
Successfully built fasttext


In [0]:
ft = fasttext.load_model('/content/drive/My Drive/Research/DataSets/wiki.pt.bin')

In [0]:
tokenizer= text.Tokenizer(num_words=5000)
tokenizer.fit_on_texts(datatrain.sent_token_final.tolist())
training_sequences = tokenizer.texts_to_sequences(datatrain.sent_token_final.tolist())
train_word_index = tokenizer.word_index
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)
######
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
ele_to_pop=[]
for word,index in train_word_index.items():
  if word in ft:
    train_embedding_weights[index,:]=ft[word]
  else:
    train_embedding_weights[index,:]= np.zeros(EMBEDDING_DIM)
    ele_to_pop.append(word)
train_embedding_weights= train_embedding_weights[~np.all(train_embedding_weights==0.0,axis=1)]
for name,key in list(train_word_index.items()):
  if name in ele_to_pop:
    del train_word_index[name]
#####
label_names = ['Risk_HIGH', 'Risk_MEDIUM-LOW']

In [0]:
model = Sequential()
model.add(Embedding(len(train_word_index), EMBEDDING_DIM, weights=[train_embedding_weights], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
model.add(Dropout(0.5))   
model.add(Bidirectional(LSTM(emdedding_size, return_sequences=True)))
#model.add(Bidirectional(LSTM(units=emdedding_size, input_shape=(MAX_SEQUENCE_LENGTH))))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(len(list(label_names)),activation='softmax'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5000, 300)         13191900  
_________________________________________________________________
dropout (Dropout)            (None, 5000, 300)         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 5000, 64)          85248     
_________________________________________________________________
dense (Dense)                (None, 5000, 256)         16640     
_________________________________________________________________
dropout_1 (Dropout)          (None, 5000, 256)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1280000)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                

In [0]:
test_sequences = tokenizer.texts_to_sequences(datatest.sent_token_final.tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

x_train = train_cnn_data
y_train = datatrain[label_names].values
y_tr = y_train

x_test= test_cnn_data
y_test= datatest[label_names].values
y_te = y_test

model.fit(x_train,y_tr, batch_size=32, epochs=10,validation_data=(x_test,y_te))

Epoch 1/10
557/557 [==============================] - 227s 407ms/step - loss: 4.6015 - accuracy: 0.7000 - val_loss: 0.5375 - val_accuracy: 0.9649
Epoch 2/10
557/557 [==============================] - 226s 406ms/step - loss: 4.6009 - accuracy: 0.7000 - val_loss: 0.5375 - val_accuracy: 0.9649
Epoch 3/10
557/557 [==============================] - 226s 406ms/step - loss: 4.6015 - accuracy: 0.7000 - val_loss: 0.5375 - val_accuracy: 0.9649
Epoch 4/10
557/557 [==============================] - 226s 405ms/step - loss: 4.6022 - accuracy: 0.7000 - val_loss: 0.5375 - val_accuracy: 0.9649
Epoch 5/10
557/557 [==============================] - 225s 405ms/step - loss: 4.6002 - accuracy: 0.7000 - val_loss: 0.5375 - val_accuracy: 0.9649
Epoch 6/10
557/557 [==============================] - 225s 405ms/step - loss: 4.6009 - accuracy: 0.7000 - val_loss: 0.5375 - val_accuracy: 0.9649
Epoch 7/10
557/557 [==============================] - 225s 405ms/step - loss: 4.6002 - accuracy: 0.7000 - val_loss: 0.5375 -

In [0]:
predictions0 = model.predict(train_cnn_data, batch_size=32, verbose=1)
predictions = model.predict(test_cnn_data, batch_size=32, verbose=1)

173/173 [==============================] - 32s 183ms/step


In [0]:
predictions_01 = np.asarray([np.argmax(x) for x in predictions0])
predictions_1 = np.asarray([np.argmax(x) for x in predictions])

In [0]:
acc, pre, rec, f1s = get_metrics(datatrain.FinalLabel,predictions_01)
acc, pre, rec, f1s = get_metrics(datatest.FinalLabel,predictions_1)

Accuracy: 0.7  Precision: 0.7  Recall: 1.0  F1 Score: 0.8235
Accuracy: 0.9649  Precision: 0.9649  Recall: 1.0  F1 Score: 0.9821


In [0]:
# accr = model.evaluate(test_cnn_data,y_te)

In [0]:
predictions_1 = np.argmax(predictions, axis=1)
print(classification_report(datatest.FinalLabel, predictions_1))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       194
           1       0.96      1.00      0.98      5337

    accuracy                           0.96      5531
   macro avg       0.48      0.50      0.49      5531
weighted avg       0.93      0.96      0.95      5531



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
